In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
from quantfreedom.levon_qf.strategy_maker_cls_levon import StrategyMaker

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    CandleBody,
)
from quantfreedom.plotting.strat_dashboard import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )
sm = StrategyMaker()
# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf('../tests/data/prices.hd5')
rsi_ind = sm.from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    cand_ohlc="close",
)
sm.combined_data_frame()

symbol              BTCUSDT                      EOSUSD                       \
RSI_timeperiod           15            30            15            30          
RSI_is_below             40     50     40     50     40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00   False  False  False  False  False  False  False  False   
2020-10-01 00:30:00   False  False  False  False  False  False  False  False   
2020-10-01 01:00:00   False  False  False  False  False  False  False  False   
2020-10-01 01:30:00   False  False  False  False  False  False  False  False   
2020-10-01 02:00:00   False  False  False  False  False  False  False  False   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2021-04-14 21:30:00   False   True  False   True  False  False  False  False   
2021-04-14 22:00:00   False   True  False   True  False  False  False  False   
2021-04-14 22:30:00   False   True  False   True  False  False  False  False   
2021-04-14 23:00:00   False   True  False   True  False  False  False  False   
2021-04-14 23:30:00   False   True  False   True  False  False  False  False   

symbol              ETHUSD                      LTCUSDT                       \
RSI_timeperiod          15            30             15            30          
RSI_is_below            40     50     40     50      40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00  False  False  False  False   False  False  False  False   
2020-10-01 00:30:00  False  False  False  False   False  False  False  False   
2020-10-01 01:00:00  False  False  False  False   False  False  False  False   
2020-10-01 01:30:00  False  False  False  False   False  False  False  False   
2020-10-01 02:00:00  False  False  False  False   False  False  False  False   
...                    ...    ...    ...    ...     ...    ...    ...    ...   
2021-04-14 21:30:00  False  False  False  False   False  False  False  False   
2021-04-14 22:00:00  False  False  False  False   False  False  False  False   
2021-04-14 22:30:00  False  False  False  False   False  False  False  False   
2021-04-14 23:00:00  False  False  False  False   False  False  False  False   
2021-04-14 23:30:00  False  False  False  False   False  False  False  False   

symbol              XRPUSD                       
RSI_timeperiod          15            30         
RSI_is_below            40     50     40     50  
open_time                                        
2020-10-01 00:00:00  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  
...                    ...    ...    ...    ...  
2021-04-14 21:30:00  False  False  False  False  
2021-04-14 22:00:00  False  False  False  False  
2021-04-14 22:30:00  False  False  False  False  
2021-04-14 23:00:00  False  False  False  False  
2021-04-14 23:30:00  False  False  False  False  

[9408 rows x 20 columns]

In [2]:
strat_results_df, settings_results_df = sm.backtest_df(
    prices=price_data,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(7, 8.1, 1),
    risk_rewards=4,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=8,
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=np.arange(3, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 4
Total order settings to test: 20
Total candles per symbol: 9,408
Total candles to test: 752,640

Total combinations to test: 80


In [3]:
strat_results_df


,symbol,entries_col,or_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
24,ETHUSD,11,2,49.00,257.21,46.94,0.86,"2,572.06","3,572.06"
22,ETHUSD,11,0,49.00,208.88,46.94,0.85,"2,088.77","3,088.77"
20,ETHUSD,10,2,35.00,251.22,48.57,0.85,"2,512.17","3,512.17"
18,ETHUSD,10,0,36.00,212.94,47.22,0.85,"2,129.45","3,129.45"
16,ETHUSD,9,2,52.00,234.41,48.08,0.85,"2,344.13","3,344.13"
25,ETHUSD,11,3,48.00,265.85,47.92,0.84,"2,658.47","3,658.47"
21,ETHUSD,10,3,34.00,261.88,50.00,0.83,"2,618.76","3,618.76"
23,ETHUSD,11,1,48.00,211.14,47.92,0.83,"2,111.35","3,111.35"
14,ETHUSD,9,0,53.00,185.99,47.17,0.83,"1,859.85","2,859.85"
17,ETHUSD,9,3,51.00,242.50,49.02,0.83,"2,425.02","3,425.02"


In [6]:
sm.select_row_and_simulate([1,22])

(symbol                BTCUSDT                                 ETHUSD           \
 candle_info              open      high       low     close     open     high   
 open_time                                                                       
 2020-10-01 00:00:00 10,759.00 10,817.50 10,759.00 10,790.00   359.85   361.45   
 2020-10-01 00:30:00 10,790.00 10,790.50 10,775.00 10,777.00   358.95   359.75   
 2020-10-01 01:00:00 10,777.00 10,793.00 10,777.00 10,793.00   359.25   359.80   
 2020-10-01 01:30:00 10,793.00 10,836.00 10,792.50 10,822.00   359.75   361.40   
 2020-10-01 02:00:00 10,822.00 10,841.50 10,808.00 10,808.50   361.20   364.05   
 ...                       ...       ...       ...       ...      ...      ...   
 2021-04-14 21:30:00 62,703.50 62,812.50 62,615.50 62,795.00 2,406.20 2,440.00   
 2021-04-14 22:00:00 62,795.00 62,825.00 62,591.00 62,680.00 2,434.05 2,445.00   
 2021-04-14 22:30:00 62,680.00 62,993.00 62,680.00 62,849.50 2,427.00 2,438.60   
 2021-04-14 23:0